https://www.kaggle.com/pcbreviglieri/lastfm-music-artist-scrobbles
Рекомендательная система для музыки.

In [3]:
import pandas as pd 
import numpy as np
from scipy import sparse 
from sklearn.preprocessing import normalize

In [5]:
intraction_df = pd.read_csv('lastfm_user_scrobbles.csv')
titles_df = pd.read_csv('lastfm_artist_list.csv')

In [6]:
intraction_df

,user_id,artist_id,scrobbles
0,1,4562,13883
1,1,10191,11690
2,1,494,11351
3,1,6673,10300
4,1,8402,8983
...,...,...,...
92787,1892,10098,278
92788,1892,8660,263
92789,1892,3274,258
92790,1892,4240,232


In [7]:
titles_df

,artist_id,artist_name
0,1,__Max__
1,2,_Algol_
2,3,-123 Min.
3,4,-Oz-
4,5,-T De Sangre
...,...,...
17488,17489,鷺巣詩郎
17489,17490,黃立行
17490,17491,黄义达
17491,17492,黒木メイサ


In [10]:
## распределение записей по пользователям 
intraction_df.groupby('user_id').count().mean()

artist_id    49.044397
scrobbles    49.044397
dtype: float64

In [13]:
## матрица user item 
titles_df.index = titles_df['artist_id']
title_dict = titles_df['artist_name'].to_dict()

In [20]:
## получение матрицы 
rows, r_pos = np.unique(intraction_df.values[:,0], return_inverse=True)
cols, c_pos = np.unique(intraction_df.values[:,1], return_inverse=True)

In [21]:
cols

array([    1,     2,     3, ..., 17491, 17492, 17493], dtype=int64)

In [22]:
c_pos

array([ 4561, 10190,   493, ...,  3273,  4239, 10841], dtype=int64)

In [24]:
# конструирование sparse матрицы 
interaction_sparse = sparse.csr_matrix((intraction_df.values[:,2], (r_pos, c_pos)))

In [25]:
interaction_sparse

<1892x17493 sparse matrix of type '<class 'numpy.int64'>'
	with 92723 stored elements in Compressed Sparse Row format>

In [26]:
## Нормализация и получение матрицы схожести 
## sim - матрица смежности графов 
Pui = normalize(interaction_sparse, norm='l2', axis=1)
sim = Pui.T * Pui

In [27]:
sim.todense()

matrix([[0.00120405, 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.00258841, 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.00383689, ..., 0.        , 0.        ,
         0.        ],
        ...,
        [0.        , 0.        , 0.        , ..., 0.0228355 , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.00827076,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.0010429 ]])

In [29]:
## проверка адекватности 
[title_dict[i+1] for i in sim[1763].toarray().argsort()[0][-20:]]

['Luis Miguel',
 'Boards Of Canada',
 'Alejandra Guzman',
 'Monica Naranjo',
 'Alanis Morissette',
 '3oh!3',
 'Shakira',
 'Atb',
 'Celine Dion',
 'Nas',
 'Tiësto',
 'Rihanna',
 'Nelly Furtado',
 'Kylie Minogue',
 'Christina Aguilera',
 'Benny Benassi',
 'Mariah Carey',
 'Britney Spears',
 'Madonna',
 'Lady Gaga']

In [33]:
## Делаем предсказания для пользователей 
interactions_spare_transposed = interaction_sparse.transpose(copy=True)
Piu = normalize(interactions_spare_transposed, norm='l2', axis=1)

fit = Pui * Piu * Pui

In [34]:
fit.shape

(1892, 17493)

In [40]:
## рекомендации
[title_dict[i+1] for i in fit[111].toarray().argsort()[0][-70:].tolist()]
pred_set = set([title_dict[i+1] for i in fit[111].toarray().argsort()[0][-70:].tolist()])

In [42]:
pred_set - init_set

26